---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [5]:
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Load and exploration

In [41]:
train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
# addr = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")

/Users/elece/miniconda3/envs/baseML/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
# msno.matrix(train)

In [43]:
# plt.figure(figsize=(17,8)) 
# train.iloc[:,1:].nunique().plot(kind='bar')

# Cleaning and dataset creation

In [44]:
train.drop(train[train.country != 'USA'].index, inplace=True)

potencial columns/features: 'ticket_id', 'agency_name', 'inspector_name', 'city', 'state', 'zip_code', 'ticket_issued_date',
'hearing_date', 'violation_code', 'violation_description', 'disposition', 'fine_amount', 'discount_amount','judgment_amount',

In [45]:
train.dropna(subset=['compliance', 'state'], inplace=True)


In [46]:
X_train = train[['agency_name', 'state', 'disposition',
                 'fine_amount', 'discount_amount', 'judgment_amount']]
y_train = train['compliance']

In [47]:
X_test = test[['ticket_id', 'agency_name', 'state', 'disposition',
               'fine_amount', 'discount_amount', 'judgment_amount']]

In [48]:
X_test.set_index('ticket_id', inplace=True)

In [60]:
X_test.isna().sum()

agency_name          0
state              331
disposition          0
fine_amount          0
discount_amount      0
judgment_amount      0
dtype: int64

what to do with NaNs in State. We can't drop rows in the test set because the length is a constrained. 
Fill with popular value?

In [59]:
X_test.state.value_counts().head()

MI    51866
CA     1877
TX      913
FL      863
NY      802
Name: state, dtype: int64

In [61]:
X_test.loc[:,'state'].fillna(value='MI', inplace=True)

/Users/elece/miniconda3/envs/baseML/lib/python3.6/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [62]:
#cadinality of categories
print([f'{i}:{X_train[i].nunique()}' for i in X_train.columns if X_train[i].dtype == 'object'])

['agency_name:5', 'state:58', 'disposition:4']


In [63]:
X_train = pd.get_dummies(X_train, prefix=['dispo', 'a_name', 'ST'], columns=[
                         'disposition', 'agency_name', 'state'], drop_first=True)

In [64]:
X_test = pd.get_dummies(X_test, prefix=['dispo', 'a_name', 'ST'], columns=[
                         'disposition', 'agency_name', 'state'], drop_first=True)

The test set produces differente dummy columns (train:67, test:69). There must be some value for a categorical column missing in the train. Let's fix it

In [65]:
X_train, X_test = X_train.align(X_test, join='right', axis=1, fill_value=0)

In [66]:
X_train.shape, X_test.shape

((159789, 69), (61001, 69))

# Model

In [67]:
grid_vals = {'n_estimators': [6,8,10,12,14,16], 'max_depth': [3, 4, 5, 6, 10]}

In [68]:
model = RandomForestClassifier()

In [69]:
grid_search = GridSearchCV(model, grid_vals, scoring='roc_auc', n_jobs=-1, cv=5)

In [70]:
grid_result = grid_search.fit(X_train, y_train)

In [71]:
grid_result.best_params_, grid_result.best_score_

({'max_depth': 10, 'n_estimators': 16}, 0.7867685474131447)

In [72]:
X_test_pred = grid_result.predict_proba(X_test)

In [150]:
X_test_pred

array([[0.93723644, 0.06276356],
       [0.97377648, 0.02622352],
       [0.93723644, 0.06276356],
       ...,
       [0.94527833, 0.05472167],
       [0.94527833, 0.05472167],
       [0.86999006, 0.13000994]])

In [73]:
pd.Series(index=X_test.index, data=grid_result.predict_proba(X_test)[:,1], name='compliance', dtype='float32')

ticket_id
284932    0.942233
285362    0.981439
285361    0.939593
285338    0.942233
285346    0.939593
            ...   
376496    0.981471
376497    0.981471
376499    0.942253
376500    0.942253
369851    0.772079
Name: compliance, Length: 61001, dtype: float32

# Rearrenge for coursera upload

In [155]:
# import missingno as msno
# import matplotlib.pyplot as plt
# %matplotlib inline
import pandas as pd
# import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


def blight_model():

    train = pd.read_csv('train.csv', encoding='ISO-8859-1')
    test = pd.read_csv('test.csv', encoding="ISO-8859-1")

    train.drop(train[train.country != 'USA'].index, inplace=True)
    train.dropna(subset=['compliance', 'state'], inplace=True)
    test.dropna(subset=['state'], inplace=True)
    X_train = train[['agency_name', 'state', 'disposition',
                     'fine_amount', 'discount_amount', 'judgment_amount']]
    y_train = train['compliance']
    X_test = test[['ticket_id', 'agency_name', 'state', 'disposition',
               'fine_amount', 'discount_amount', 'judgment_amount']]
    X_test.set_index('ticket_id', inplace=True)

    X_train = pd.get_dummies(X_train, prefix=['dispo', 'a_name', 'ST'], columns=[
                             'disposition', 'agency_name', 'state'], drop_first=True)

    X_test = pd.get_dummies(X_test, prefix=['dispo', 'a_name', 'ST'], columns=[
        'disposition', 'agency_name', 'state'], drop_first=True)

    X_train, X_test = X_train.align(X_test, join='right', axis=1, fill_value=0)

    grid_vals = {'n_estimators': [6, 8, 10, 12,
                                  14, 16], 'max_depth': [3, 4, 5, 6, 10]}
    model = RandomForestClassifier()
    grid_search = GridSearchCV(
        model, grid_vals, scoring='roc_auc', n_jobs=-1, cv=5)
    grid_result = grid_search.fit(X_train, y_train)
    X_test_pred = grid_result.predict_proba(X_test)
    ans = pd.Series(index=X_test.index, data=grid_result.predict_proba(
        X_test)[:, 0], name='compliance', dtype='float32')

    return ans

In [156]:
blight_model()

ticket_id
284932    0.935162
285362    0.977349
285361    0.934372
285338    0.935162
285346    0.934372
            ...   
376496    0.980717
376497    0.980717
376499    0.938121
376500    0.938121
369851    0.712982
Name: compliance, Length: 60670, dtype: float32